## 传递部分参数

operator.add是一个接收两个参数的函数，我们希望基于它快速实现一个“加一”操作，即得到一个接收一个参数的函数。

In [1]:
import operator

operator.add(1, 2)

3

最质朴的实现是：

In [2]:
def add_one(num):
  return operator.add(1, num)

利用前一节嵌套函数和闭包的知识，我们很快能得到一个更灵活的实现：

In [3]:
def create_adder(num):
  def adder(n):
    return operator.add(n, num)
  return adder


add_one = create_adder(1)
add_one(2)

3

如果要基于`operator.mul`实现一个类似的“乘二”操作，上面的`create_adder`又显得不太灵活了。于是我们创建一个更灵活的版本：

- 接收一个二元函数`f`和其中一个参数`a`
- 返回一个一元函数`g`，调用这个一元函数`g(b)`时，执行`g(a, b)`

In [4]:
def currying(f, a):
  def g(b):
    return f(a, b) 
  return g


add_one = currying(operator.add, 1)
double = currying(operator.mul, 2)


print(add_one(3))
print(double(4))

4
8


Python的functools库已经实现了类似的柯里化方法，称为`functools.partial`。partial的传参比我们自己实现的currying更灵活，支持多个参数传参或者按名字（keyword）传参。

In [5]:
import functools

add_one = functools.partial(operator.add, 1)
double = functools.partial(operator.mul, 2)

print(add_one(3))
print(double(4))

4
8


柯里化或者Partial技术隐藏了闭包实现，节省了我们定义嵌套函数的时间。如上一节实现的日期时间格式化函数：

```python
def date_time_formatter(sep, default):
  def format(date, time):
    if date is None or time is None:
      return default
    return date + sep + time
  return format


my_formatter = date_time_formatter('/', 'N/A')

print(my_formatter('2022-01-01', '13:32'))
print(my_formatter('2022-01-01', None))
```

其中用到了一个嵌套函数。但使用Partial技术，我们可以简化实现，回归最简单的函数设计：

In [6]:
def date_time_formatter(date, time, sep=" ", default="Unknown"):
  if date is None or time is None:
    return default
  return date + sep + time


my_formatter = functools.partial(date_time_formatter, sep='/', default='N/A')

print(my_formatter('2022-01-01', '13:32'))
print(my_formatter('2022-01-01', None))

2022-01-01/13:32
N/A


## 真正的柯里化

把接受N个参数的函数变换成接受一个单一参数（最初函数的第一个参数）的函数，并且返回接受余下的N-1个参数而且返回结果的新函数的技术。

一般认为柯里化是函数的性质，而不是一个额外的函数，即函数自身应该满足下面的性质：

```python

# GIVEN
def f(a, b):
  ...

# AND
g = f(x)

# THEN
f(x, y) == g(y)
```

Python里的函数并不是天然柯里化的，但toolz提供了对应的工具：

In [7]:
import toolz


@toolz.curry
def curried_date_time_formatter(date, time, sep=" ", default="Unknown"):
  if date is None or time is None:
    return default
  return date + sep + time


my_curried_formatter = curried_date_time_formatter(sep='/', default='N/A')

print(my_curried_formatter('2022-01-01', '13:32'))
print(my_curried_formatter('2022-01-01', None))

2022-01-01/13:32
N/A
